<a href="https://colab.research.google.com/github/AlejandroOliverosVera/blank/blob/main/FS_iCSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random as rnd
import math
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA
import sklearn.gaussian_process as gp
import tensorflow as tf
import time as tm

In [ ]:
class ML:
  def __init__(self):
    # Leer el archivo CSV
    self.data = pd.read_csv('/content/drive/MyDrive/data.csv', header = 0)

  def load_data(self, selected_features):
    # Usa las caracteristicas dadas por la MH
    all_features = list(self.data.columns)
    selected_column_names = [all_features[i] for i in range(len(selected_features)) if selected_features[i] == 1]
    return self.data[selected_column_names].values, self.data.iloc[:, -1].values.astype(float)


In [ ]:
class MLSVM(ML):
  def train(self, selected_features):
    X, y = self.load_data(selected_features)

    # Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Normalizar características
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entrenar un clasificador SVM
    svm = SVC()
    svm.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = svm.predict(X_test)

    # Calcular las métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average = 'weighted')

    return [accuracy, f1]

In [ ]:
class Crow:
  def __init__(self):
    self.dimension = 755 ## Depende del DSet
    self.position = []
    self.metrics = [0,0]
    self.pBest = []
    self.fBest = 0.0
    for j in range(self.dimension):
      self.position.append( )

  def countCols(self):
    sum = 0
    for j in range(self.dimension):
        sum += self.position[j]
    return sum

  def train(self, ml):
    self.metrics = ml.train(self.position)

  def is_better_than_pbest(self):
    if self.fitness() > self.fBest:
        self.fBest = self.fitness()
        return True
    else:
        return False

  def update_pbest(self):
        self.pBest = self.position[:]

  def isBetterThan(self, gBest):
    return self.fitness() > gBest.fitness()

  def move(self, followed_crow: "Crow", AP: float, flight_length: float):
    for j in range(self.dimension):
      if rnd.uniform(0, 1) >= AP:
        self.position[j] = int(self.position[j] + rnd.uniform(0, 1) * flight_length * (followed_crow.pBest[j] - self.position[j]))
      else:
        self.position[j] = rnd.randint(0, 1)

  def fitness(self):
    z1 = self.metrics[0] # maximizar f1 score -> [0: malo, 1: bueno]
    z2 = self.metrics[1] # maximizar acc -> [0: malo, 1: bueno]
    z3 = self.countCols() / self.dimension # minimizar cantidad de columnas que se usan -> [0: bueno, 1: malo]
    return z1 * 0.4 + z2 * 0.4 - z3 * 0.2 + 0.2

  def copy(self, other: "Crow"):
    self.position = other.position.copy()
    self.fBest = other.fBest

  #def toBinary(self, x):
    #return 1 if rnd.uniform(0, 1) <= 1 / (1 + math.exp(-x)) else 0


In [ ]:
class CSA:
  def __init__(self):
    self.T = 50 #iterMax
    self.nCrows = 50 #flock size
    self.AP = 0.5
    self.flightLenght = 0.75
    self.swarm = [] # array of Crow objects
    self.gCrow = Crow() #Aquí se guarda el mejor cuervo
    self.rnd = rnd
    self.sTime = None
    self.eTime = None

  def execute(self, ml):
    self.startTime()
    self.init(ml)
    self.run(ml)
    self.endTime()
    self.log()

  def startTime(self):
    self.sTime = int(round(tm.time() * 1000))

  def init(self, ml):
    for i in range(self.nCrows):
      self.swarm.append(Crow())
      self.swarm[i].train(ml)
    self.gCrow.copy(self.swarm[0])
    for i in range(1, self.nCrows):
      if self.swarm[i].isBetterThan(self.gCrow):
        self.gCrow.copy(self.swarm[i])

  def run(self, ml):
    t = 1
    while t <= self.T:
      for i in range(self.nCrows):
        if self.swarm[i].is_better_than_pbest():
            self.swarm[i].update_pbest()
        if self.swarm[i].isBetterThan(self.gCrow):
            self.gCrow.copy(self.swarm[i])
      for i in range(self.nCrows):
        followedCrow = self.swarm[self.rnd.randint(0, self.nCrows - 1)]
        self.swarm[i].move(followedCrow, self.AP, self.flightLenght)
        self.swarm[i].train(ml)
      t += 1

  def endTime(self):
    self.eTime = int(round(tm.time() * 1000))

  def log(self):
    print(f"Mejor fitness: {self.gCrow.fBest}, Tiempo total: {self.eTime - self.sTime}ms")


In [ ]:
try:
  CSA().execute(MLSVM())
except Exception as e:
  print(f"{e} \nCaused by {e.args}")


Mejor fitness: 0.9406853089518381, Tiempo total: 91713ms
